In [1]:
#  word level CTC  tensorflow version 1.3

# this module requires the following :

#  a dataset csv file ( have a look at dataset csv generator)
#  a dictionary.txt file ( have a look at dictionary folder)



# to do :   
#  , testing on realdataset , get rid of fixed batch size , instead use variable batch size  


# perform following tests :

# 1. variable batch sizes test
# 2. GPU test





# please change the following according to your system
# hyperparameters in this notebook

# change paths in this notebook to that corresponding to your system
# please note number of audio examples in dataset must be even 

# peace.




In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [5]:
inputs.shape

(299, 13)

In [6]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [13]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("/home/saurabh/Downloads/audio2/words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i



In [14]:
print(len(word_dictionary))

492


In [15]:



def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [16]:
word_to_index('this is a test')

[485]

In [17]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [18]:
def _read_py_function(audio, label):
    audio = audio_to_mfcc(audio)
    
    if audio.shape[0] < timesteps:
        original_length=audio.shape[0]
       # print(original_length)
    
    elif audio.shape[0] >= timesteps:
        original_length=timesteps
        
    
    audio=pad(audio)
   
    return audio ,label, original_length

In [19]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/hardik.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype, tf.int64])))




dataset=dataset.batch(2)


In [20]:

iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [21]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [22]:
# Training Parameters
learning_rate = 0.001
training_steps = 100
#batch_size = 2
display_step = 200
num_features = 13
logs_path = '/home/saurabh/Documents/tf_orange/tf_orange/hardik/logs'
model_path = '/home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt'

# Network Parameters

timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = len(word_dictionary) + 1 # words.txt , all words plus space word  ( 10001) + CTC symbol (1)




In [23]:
#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])
batch_size = 2

#batch_size = tf.placeholder ( tf.int32 )


In [24]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([ 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [25]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,2*num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [26]:




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [27]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [28]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [29]:
init = tf.global_variables_initializer()

In [30]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [31]:
# testing above function 

print(keys_of_value(word_dictionary, 2419))

None


In [34]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, word_to_index(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
       
                    str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
                    print(str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk
    #save_path = saver.save(sess, model_path)
    #print("Model saved in file: %s" % save_path)         

               
        
            
                
                
             
                
                
            

Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
[b'Paris attacks suspect' b'Nokia 7 Plus Specifications']
[279 300]
 7 7 7 7 
 7 7 7 7 7 7 7 7 7 7 7 7 7 
[b'Pakistani journalist covers his own wedding'
 b'Electric vehicle scene still sketchy']
[300 300]
 
 
[b'million stake sale in Tata Technologies' b'French delegation']
[300 247]
 
 help 
[b"a solution to a problem that doesn't exist"
 b'judges facing intimidation']
[300 300]
 
 Twist TwistFabindia7 PakistanirecordnormalSyndrome
[b'ill kids loved puts life into perspective' b'Turkish President Recep']
[300 279]
 
 
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
 
 invest 
[b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'Meditation has limited role in making you a better person']
[300 300]
 
 
[b'Singtel to invest in Bharti Telecom' b'Are you over 50?']
[300 247]
 
 shipping on 
[b'diesel at all-time high in Delhi'
 b"China's success in c

 
 
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
index MoonannoyingWhatsAppcan Hyderabad Treatment 
 technique Are on 
[b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'Meditation has limited role in making you a better person']
[300 300]
 
reducingMoon MoonWhatsApp 
[b'Singtel to invest in Bharti Telecom' b'Are you over 50?']
[300 247]
 
 reducing fast Redmi 
[b'diesel at all-time high in Delhi'
 b"China's success in cloning monkeys was a victory for hard work"]
[300 300]
 
 
[b'Chinese Incursions Into India Rose'
 b'Cancer Treatment Could Increase Heart Disease Risk']
[300 300]
 
 Hyundai U12'14 14 UK 
[b"Rover to spend 14 days on moon's surface"
 b'49% of Iranians against compulsory veil']
[300 300]
 Flexible annoying Into fast 
 
[b'Khadi commission seeks compensation from Fabindia'
 b'SpaceX Falcon Heavy Poised For Debut Test Launch']
[300 300]
 Theory firmU12' launchesmaking 
TheoryannoyingFlexible 
[b'Indian-Americans hold

 Flexible HyderabadInocean
 Earth making 
[b'These 4 lifestyle modifications can help you FIGHT cancer!'
 b'Hello Airtel postpaid users']
[300 300]
Earth sale 
reducingU12' 
[b'Computer Vision Syndrome' b'Cholera alert in State']
[300 279]
 
 
[b'Apple raises iPhone prices in India' b'Who are your friends?']
[300 247]
 
Earth focus fast Southare inThe
[b'Hyundai Elite i20 facelift India launch confirmed at Auto Expo'
 b'Long screen exposure at work']
[300 300]
 
U12' in 
[b'CBI books billionaire' b'Donald Trump over NHS comments']
[294 300]
 sold 
 
[b'Time has come to make a Brexit choice' b'Disturbing trend']
[300 263]
 Renault antibiotic 
 
[b'announcements may help you save tax'
 b'Pakistan government to arrest him']
[300 300]
 
 hold was Race
[b'speech sparks backlash'
 b"Indian scientists develop world's thinnest material with novel technique"]
[300 300]
 
 criticism 
[b'iBall CompBook Premio' b"'HTC U12' With 5G Support"]
[300 300]
maritime PakistanCompBook South compensation Di

CBI to
galaxy ontalks 
[b'Bitcoin newbies are getting crushed'
 b'Three parent babies business as usual?']
[300 300]
CBI Lunar be government 
 be64Google talks talks Expoas 
[b'Jeep Compass Trailhawk SUV' b'Inner Ear Helps Cheetah Run Fast']
[300 300]
 sold sold confirmed 
 Helps 
[b'Trade setup for Tuesday' b'Intel made smart glasses that look normal']
[294 300]
 modifications for 'Potentiallynot
your 
[b'US ban Bitcoin buying with credit cards'
 b'Tata Motors Q3 profit surges']
[300 300]
 Theory
 hard  
[b"South Africa's ANC holds crunch talks"
 b"Vivo V7+ Infinite Red 'Limited Edition' Launched in India"]
[300 300]
 antibiotic 
 Edition' kidsimpactlaunchedthe 
[b"Whales are at 'significant' risk from ocean microplastic"
 b'These Two Firms Are In Race']
[300 300]
 be Run 
galaxy 
[b'Dance of galaxies challenges current thinking on cosmology'
 b'PM Modi to focus on maritime relations']
[300 300]
 
 
[b'Twist in out-of-Africa theory'
 b'Sony Xperia L2 With Wide-Angle Selfie Camera']
[3

'should Google 
Darwin's S9 of 
[b'Niti Aayog health index' b'Chinese shipping firm MD killed']
[300 300]
'shouldchoice 'illegal' 
 
[b'community members visit India'
 b'Flexible battery inspired by human spine']
[300 300]
 making to Africa'svisit Natural
Rose wedding prices
7548.333068847656
Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
[b'Paris attacks suspect' b'Nokia 7 Plus Specifications']
[279 300]
maritime suspect
Darwin's 
[b'Pakistani journalist covers his own wedding'
 b'Electric vehicle scene still sketchy']
[300 300]
 Darwin's 
warranty 5 
[b'million stake sale in Tata Technologies' b'French delegation']
[300 247]
 save 
killed Rover be annoying
[b"a solution to a problem that doesn't exist"
 b'judges facing intimidation']
[300 300]
 
 rocket
[b'ill kids loved puts life into perspective' b'Turkish President Recep']
[300 279]
 
 For
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
 can 
 L2 
[b"A conc

...

 Flips Switch on Its Core
Oil continues to missing
[b'Natural remedies to get rid of that annoying migraine pain'
 b"64 percent of antibiotic cocktails sold in India 'illegal'"]
[300 300]
Muslim to get rid that annoying migraine pain
Super antibiotic cocktails sold in India 'illegal'
[b'Planets beyond Milky Way galaxy discovered for first time'
 b'States not in favour of petrol']
[300 300]
 beyond Milky discovered for time
government in favour of petrol
[b'Niti Aayog health index' b'Chinese shipping firm MD killed']
[300 300]
Planets health 
 shipping firm MD 
[b'community members visit India'
 b'Flexible battery inspired by human spine']
[300 300]
Man members visit India
World's community inspired by human spine
911.4249544143677
Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
[b'Paris attacks suspect' b'Nokia 7 Plus Specifications']
[279 300]
Paris attacks suspect
of Plus Specifications
[b'Pakistani journalist covers his own wedding'
 b'Elect

 beyond Milkysketchy Way galaxy discovered for time
government in favour of petrol
[b'Niti Aayog health index' b'Chinese shipping firm MD killed']
[300 300]
Planets health 
 shipping firm MD 
[b'community members visit India'
 b'Flexible battery inspired by human spine']
[300 300]
 members visit India
World's community inspired by human spine
887.553352355957
Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
[b'Paris attacks suspect' b'Nokia 7 Plus Specifications']
[279 300]
Paris suspect
of Plus Specifications
[b'Pakistani journalist covers his own wedding'
 b'Electric vehicle scene still sketchy']
[300 300]
 covers his own wedding
Electric 2018 scene 
[b'million stake sale in Tata Technologies' b'French delegation']
[300 247]
Firms stake sale in Tata Technologies
French delegation
[b"a solution to a problem that doesn't exist"
 b'judges facing intimidation']
[300 300]
a solution to problem that doesn't 
 intimidation
[b'ill kids loved puts life

Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
[b'Paris attacks suspect' b'Nokia 7 Plus Specifications']
[279 300]
Paris attacks suspect
of Plus Specifications
[b'Pakistani journalist covers his own wedding'
 b'Electric vehicle scene still sketchy']
[300 300]
 covers his own wedding
Electric 2018 scene 
[b'million stake sale in Tata Technologies' b'French delegation']
[300 247]
Firms stake sale in Tata Technologies
French delegation
[b"a solution to a problem that doesn't exist"
 b'judges facing intimidation']
[300 300]
a solution to problem that doesn't 
 facing intimidation
[b'ill kids loved puts life into perspective' b'Turkish President Recep']
[300 279]
 kids puts life perspective
 President Recep
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
Cashews can improve good cholesterol levels
Samsung Galaxy S9
[b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'Meditation has limited role in 

 covers his own wedding
Electric vehicle scene 
[b'million stake sale in Tata Technologies' b'French delegation']
[300 247]
Firms stake sale in Tata Technologies
French delegation
[b"a solution to a problem that doesn't exist"
 b'judges facing intimidation']
[300 300]
a solution to problem that doesn't 
 facing intimidation
[b'ill kids loved puts life into perspective' b'Turkish President Recep']
[300 279]
 kids puts life perspective
 President Recep
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
Cashews can improve good cholesterol levels
 Galaxy S9
[b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'Meditation has limited role in making you a better person']
[300 300]
A look Darwin's and its criticism
 role in making you a better person
[b'Singtel to invest in Bharti Telecom' b'Are you over 50?']
[300 247]
 to in Bharti Telecom
Are you over 50?
[b'diesel at all-time high in Delhi'
 b"China's success in cloning monkeys was a victory

a solution to problem that doesn't 
 facing intimidation
[b'ill kids loved puts life into perspective' b'Turkish President Recep']
[300 279]
 kids puts life perspective
 President Recep
[b'Cashews can improve good cholesterol levels' b'Samsung Galaxy S9']
[300 300]
Cashews can improve good cholesterol levels
Samsung Galaxy S9
[b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'Meditation has limited role in making you a better person']
[300 300]
A look Darwin's and its criticism
 role in making you a better person
[b'Singtel to invest in Bharti Telecom' b'Are you over 50?']
[300 247]
 to in Bharti Telecom
Are you over 50?
[b'diesel at all-time high in Delhi'
 b"China's success in cloning monkeys was a victory for hard work"]
[300 300]
speech at all-time high in Delhi
 success in a victory for hard 
[b'Chinese Incursions Into India Rose'
 b'Cancer Treatment Could Increase Heart Disease Risk']
[300 300]
Boeing Into India 
Cancer Treatment Could Increase Heart Disease 